In [1]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [2]:
import json
import math
import multiprocessing
import random
from typing import List, Set

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
from penguinml.utils.timer import Timer
from tqdm import tqdm

import whoosh_utils
from const import ALL_KEYS, CPC2TOKENS_PATH, INF, KEY2QUERY, NUM_CPU
from sa import SimulatedAnnealing
from utils import compute_ap, evaluate, load_list_bz2, save_list_bz2

Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl
Whoosh is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [ ]:
nn_df = pl.read_csv("/kaggle/input/uspto-boolean-search-optimization/nearest_neighbors.csv")
nn_df.head(1)

publication_number,neighbor_0,neighbor_1,neighbor_2,neighbor_3,neighbor_4,neighbor_5,neighbor_6,neighbor_7,neighbor_8,neighbor_9,neighbor_10,neighbor_11,neighbor_12,neighbor_13,neighbor_14,neighbor_15,neighbor_16,neighbor_17,neighbor_18,neighbor_19,neighbor_20,neighbor_21,neighbor_22,neighbor_23,neighbor_24,neighbor_25,neighbor_26,neighbor_27,neighbor_28,neighbor_29,neighbor_30,neighbor_31,neighbor_32,neighbor_33,neighbor_34,neighbor_35,neighbor_36,neighbor_37,neighbor_38,neighbor_39,neighbor_40,neighbor_41,neighbor_42,neighbor_43,neighbor_44,neighbor_45,neighbor_46,neighbor_47,neighbor_48,neighbor_49
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""US-1-A""","""US-189152-A""","""US-1189501-A""","""US-409430-A""","""US-421894-A""","""US-9052-A""","""US-435641-A""","""US-247802-A""","""US-566579-A""","""US-148439-A""","""US-488484-A""","""US-1003894-A""","""US-951842-A""","""US-965846-A""","""US-733217-A""","""US-1681171-A""","""US-339-A""","""US-1206147-A""","""US-1213323-A""","""US-9716-A""","""US-882618-A""","""US-1161812-A""","""US-6321-A""","""US-3267876-A""","""US-1136947-A""","""US-1097810-A""","""US-939423-A""","""US-986261-A""","""US-975048-A""","""US-95968-A""","""US-5532-A""","""US-393896-A""","""US-156138-A""","""US-144271-A""","""US-357649-A""","""US-1182286-A""","""US-1342938-A""","""US-1329037-A""","""US-42039-A""","""US-3237572-A""","""US-462944-A""","""US-1108807-A""","""US-1393935-A""","""US-45450-A""","""US-381847-A""","""US-1354714-A""","""US-661045-A""","""US-737513-A""","""US-447255-A""","""US-1510390-A""","""US-1075814-A"""


In [ ]:
with open("/kaggle/input/token-counts/token_counts.json", "r") as f:
    token_counts = json.load(f)

In [ ]:
def load_patent(patent):
    try:
        path1 = os.path.join("/kaggle/input/all-index-per-patent/data", patent + ".json.bz2")
        if os.path.exists(path1):
            return load_list_bz2(path1)
        path2 = os.path.join("/kaggle/input/all-index-per-patent/data2", patent + ".json.bz2")
        if os.path.exists(path2):
            return load_list_bz2(path2)
        # print(f"Cannot find {patent}")
    except ValueError:
        pass
    return None

In [ ]:
!rm -rf queries0
!rm -rf queries1
!mkdir queries0
!mkdir queries1

In [ ]:
import random
from collections import Counter

THRESHOLD = 300
N_QUERIES = 100


def process(patents):
    patents = list(patents)
    center_patent = patents[0]
    patents = patents[1:]

    queries = []
    query2patents = {}
    query2total = {}
    for patent in patents:
        data = load_patent(patent)
        if data is None:
            queries.append([])
            continue

        this_queries = []
        for key in ALL_KEYS:
            for token in data[key]:
                if token_counts[key][token] < THRESHOLD:
                    query = KEY2QUERY[key] + ":" + token
                    this_queries.append(query)
                    query2total[query] = token_counts[key][token]

        this_queries = list(set(this_queries))
        for query in this_queries:
            if query not in query2patents:
                query2patents[query] = []
            query2patents[query].append(patent)
        queries.append(this_queries)
    assert len(queries) == len(patents)

    counts = Counter(sum([list(q) for q in queries], [])).most_common()
    query_inner_outer_patents = []
    for query, count in counts:
        n_inner = count
        n_outer = query2total[query] - n_inner
        assert len(query2patents[query]) == n_inner
        if n_inner == 1:
            continue

        query_inner_outer_patents.append((query, n_inner, n_outer, set(query2patents[query])))

    query_inner_outer_patents = sorted(
        query_inner_outer_patents,
        key=lambda x: x[1] - x[2] / 50,  # inner - outer/50
        reverse=True,
    )
    save_qury_inner_outer_patents = []
    used_patents = []
    for use_query, n_inner, n_outer, patents in query_inner_outer_patents:
        ok = True
        for used in used_patents:
            if patents.issubset(used):
                ok = False
                break
        if ok:
            save_qury_inner_outer_patents.append((use_query, n_inner, n_outer, list(patents)))
            used_patents.append(patents)

        if len(save_qury_inner_outer_patents) == N_QUERIES:
            break

    suffix = hash(center_patent) % 2
    output_file = f"queries{suffix}/{center_patent}.json.bz2"
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    save_list_bz2(save_qury_inner_outer_patents, output_file)


with multiprocessing.Pool(processes=30) as pool:
    list(tqdm(pool.imap(process, nn_df.iter_rows()), total=len(nn_df)))

  0%|          | 0/13307647 [00:00<?, ?it/s]

Cannot find US-1185281-A


  0%|          | 479/13307647 [00:04<72:48:29, 50.77it/s] 

Cannot find US-9365762-B1


  0%|          | 4845/13307647 [00:31<56:30:42, 65.39it/s] 

...

  7%|▋         | 925681/13307647 [1:42:24<15:54:30, 216.20it/s]

Cannot find US-641865-A


  7%|▋         | 928774/13307647 [1:42:44<19:00:20, 180.92it/s]

Cannot find US-7677116-B2


  7%|▋         | 931249/13307647 [1:43:01<24:08:18, 142.42it/s]